In [ ]:
import schedule
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

# Hàm lấy dữ liệu mới từ trang web và lưu vào Excel
def fetch_and_save_data():
    all_data = []
    base_url = 'https://www.scrapethissite.com/pages/forms/'

    # Kiểm tra xem file Excel đã tồn tại chưa
    try:
        # Đọc file Excel hiện tại nếu có
        df_existing = pd.read_excel(r'D:\python\File\all_table_data.xlsx')
        last_update_date = df_existing['Ngày cập nhật'].max()  # Lấy ngày cập nhật cuối cùng
        print(f"Ngày cập nhật cuối cùng là: {last_update_date}")
    except FileNotFoundError:
        # Nếu không có file Excel, sử dụng một ngày giả định ban đầu (ví dụ: ngày đầu tiên)
        last_update_date = datetime(2000, 1, 1)
        print("File Excel không tồn tại. Sử dụng ngày giả định.")

    # Lấy nội dung trang đầu tiên để xác định số trang tối đa
    response = requests.get(base_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        pagination = soup.find('ul', class_='pagination')

        if pagination:
            pages = pagination.find_all('li')
            total_pages = int(pages[-2].text.strip())  # Số trang tối đa
        else:
            print("Không tìm thấy phần phân trang.")
            total_pages = 1
    else:
        print("Không thể truy cập trang chính.")
        total_pages = 1
    
    # Duyệt qua từng trang và thu thập dữ liệu
    for page in range(1, total_pages + 1):
        url = f'{base_url}?page_num={page}'
        response = requests.get(url)

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            table = soup.find('table', class_='table')

            if table:
                rows = table.find_all('tr')

                # Lấy tiêu đề của bảng từ trang đầu tiên
                if page == 1:
                    headers = [header.text.strip() for header in rows[0].find_all('th')]
                
                # Lặp qua từng hàng của bảng (trừ hàng tiêu đề)
                for row in rows[1:]:
                    cols = row.find_all('td')
                    data_row = [col.text.strip() for col in cols]
                    data_row.append(datetime.now().strftime('%Y-%m-%d %H:%M:%S'))  # Thêm cột ngày cập nhật
                    all_data.append(data_row)

                print(f"Đã lấy dữ liệu từ trang {page}")
            else:
                print(f"Không tìm thấy bảng trong trang {page}")
        else:
            print(f"Không thể truy cập trang {page}")
    
    # Chuyển đổi danh sách dữ liệu thành DataFrame
    df_new = pd.DataFrame(all_data, columns=headers + ['Ngày cập nhật'])

    # Loại bỏ các bản ghi đã có trong file Excel (dựa trên cột 'Ngày cập nhật')
    if not df_existing.empty:
        df_combined = pd.concat([df_existing, df_new])
        df_combined = df_combined[df_combined['Ngày cập nhật'] > last_update_date]
    else:
        df_combined = df_new

    # Lưu dữ liệu vào file Excel
    df_combined.to_excel(r'D:\python\File\all_table_data.xlsx', index=False)
    print("Dữ liệu đã được lưu vào file Excel.")

# Đặt lịch để chạy hàm mỗi ngày (hoặc mỗi giờ, mỗi phút tùy ý)
schedule.every().day.at("00:00").do(fetch_and_save_data)  # Cập nhật mỗi ngày lúc 00:00 AM

# Lặp lại liên tục để kiểm tra xem có công việc nào cần chạy không
while True:
    schedule.run_pending()  # Kiểm tra các công việc đã lên lịch
    time.sleep(60)  # Kiểm tra lại sau mỗi 60 giây
